In [6]:
import numpy
import pandas
import matplotlib.pyplot as plt
import seaborn
import scipy.stats

import sklearn.linear_model
import sklearn.model_selection
import sklearn.metrics
import sklearn.preprocessing

import torch.nn as nn
import torch.optim as optim
import torch as ptr

In [ ]:
# we are desinging a BI-LSTM model to predict the stock price of a company
# we will use the past 60 days stock price to predict the next day stock price
# we will use the stock price of ALT.csx for this task
 
# load the data
data = pandas.read_csv('data/ALT (1).csv')
data = data.dropna()
data = data[['open', 'high', 'low', 'close','volume','trade_count']]
data = data.values

# split the data into training and testing data
train_data = data[:int(len(data)*0.8)]
test_data = data[int(len(data)*0.8):]

# scale the data
scaler = sklearn.preprocessing.MinMaxScaler()
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)

# create the training data
X_train = []
y_train = []
for i in range(60, len(train_data)):
    X_train.append(train_data[i-60:i])
    y_train.append(train_data[i])
X_train = numpy.array(X_train)
y_train = numpy.array(y_train)

# create the testing data
X_test = []
y_test = []
for i in range(60, len(test_data)):
    X_test.append(test_data[i-60:i])
    y_test.append(test_data[i])
X_test = numpy.array(X_test)
y_test = numpy.array(y_test)

# create the model
# Create a Bidirectional LSTM model with 5 Bi directional LSTM layers with 128, units for 3 layers and 2  layers with 64 units
# the output layer will have 4 units
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size*2, num_layers-1, batch_first=True,bidirectional=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        h0 = ptr.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = ptr.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out
    
device = ptr.device('cuda' if ptr.cuda.is_available() else 'cpu')
model = LSTM(6, 128, 6, 4).to(device)

model.compile()
print(model)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)





LSTM(
  (lstm1): LSTM(6, 256, num_layers=6, batch_first=True, bidirectional=True)
  (lstm2): LSTM(256, 128, num_layers=6, batch_first=True, bidirectional=True)
  (lstm3): LSTM(256, 128, num_layers=6, batch_first=True, bidirectional=True)
  (lstm4): LSTM(256, 64, num_layers=6, batch_first=True, bidirectional=True)
  (lstm5): LSTM(128, 64, num_layers=6, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=128, out_features=4, bias=True)
)


In [19]:
import torch._dynamo
torch._dynamo.config.suppress_errors = True

# train the model

X_train = ptr.tensor(X_train).float().to(device)
y_train = ptr.tensor(y_train).float().to(device)
X_test = ptr.tensor(X_test).float().to(device)
y_test = ptr.tensor(y_test).float().to(device)
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    outputs = model(X_train)
    optimizer.zero_grad()
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f'epoch {epoch}: loss {loss.item()}')
        
# test the model
model.eval()
outputs = model(X_test)
loss = criterion(outputs, y_test)
print(f'test loss {loss.item()}')

# plot the results
plt.figure(figsize=(14, 5))
plt.plot(scaler.inverse_transform(y_test.cpu().detach().numpy()), label='True')
plt.plot(scaler.inverse_transform(outputs.cpu().detach().numpy()), label='Predicted')
plt.legend()
plt.show()

/tmp/ipykernel_10281/1642401872.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = ptr.tensor(X_train).float().to(device)
/tmp/ipykernel_10281/1642401872.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = ptr.tensor(y_train).float().to(device)
/tmp/ipykernel_10281/1642401872.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = ptr.tensor(X_test).float().to(device)
/tmp/ipykernel_10281/1642401872.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourc

AttributeError: 'LSTM' object has no attribute 'lstm'